<a href="https://colab.research.google.com/github/noushsuon/honours-project/blob/main/NoPretraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install comet-ml
!pip install datasets
!pip install evaluate
!pip install accelerate -U
!pip install codecarbon
!pip install transformers
!pip install tokenizers

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
import comet_ml
from comet_ml import Experiment
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

experiment = Experiment(api_key="uyaIbaxDfy7jK9j7u4KNxeKNg",project_name="Honours",)
comet_ml.init(api_key='uyaIbaxDfy7jK9j7u4KNxeKNg',project_name='comet-bert-experiment')

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
[codecarbon INFO @ 17:45:26] [setup] RAM Tracking...
[codecarbon INFO @ 17:45:26] [setup] GPU Tracking...
[codecarbon INFO @ 17:45:26] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 17:45:26] [setup] CPU Tracking...
[codecarbon WARNING @ 17:45:26] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 17:45:27] We saw that you have a 13th Gen Intel(R) Core(TM) i9-13950HX but we don't know it. Please contact us.
[codecarbon INFO @ 17:45:27] CPU Model on constant consumption mode: 13th Gen Intel(R) Core(TM) i9-13950HX
[codecarbon INFO @ 17:45:27] >>> Tracker's metadata:
[codecarbon INFO @ 17:45:27]   Platform system: Linux-5.15.133.1-microsoft-standard-WSL2-x86_64-with-glibc2.35
[codecarbon INFO @ 17:45:27]   Python version: 3.10.12
[codecarbon INFO @ 17:45:27]   CodeCarbon version: 2.

In [4]:
from datasets import load_dataset
import transformers

raw_datasets = load_dataset("squad")
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [5]:
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizerFast
import os
combined_data = [context + " " + question for context, question in zip(raw_datasets['train']['context'], raw_datasets['train']['question'])]
tokenizer = BertWordPieceTokenizer(clean_text=True,lowercase=False,handle_chinese_chars=False)
tokenizer.train_from_iterator(combined_data, length=512, special_tokens=["[UNK]", "[SEP]", "[CLS]", "[PAD]", "[MASK]"])
path = "/content"
if not os.path.exists(path):
    os.makedirs(path)
tokenizer.save_model(path)
tokenizer = BertTokenizerFast.from_pretrained(path, model_max_length=512, do_lower_case=False)

[codecarbon INFO @ 17:45:42] Energy consumed for RAM : 0.000024 kWh. RAM Power : 5.789747714996338 W
[codecarbon INFO @ 17:45:42] Energy consumed for all GPUs : 0.000024 kWh. Total GPU Power : 5.777232372035104 W
[codecarbon INFO @ 17:45:42] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 17:45:42] 0.000225 kWh of electricity used since the beginning.


In [6]:
tokenizer

BertTokenizerFast(name_or_path='/content', vocab_size=30000, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [7]:
context = raw_datasets["train"][1902]["context"]
question = raw_datasets["train"][1902]["question"]

inputs = tokenizer(question, context)
tokenizer.decode(inputs["input_ids"])

"[CLS] What television station made a documentary on Chopin? [SEP] Chopin's life was covered in a BBC TV documentary Chopin – The Women Behind The Music ( 2010 ), and in a 2010 documentary realised by Angelo Bozzolini and Roberto Prosseda for Italian television. [SEP]"

In [8]:
max_length = 384
stride = 128


def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second", #only truncates the context
        stride=stride,
        return_overflowing_tokens=True, #model will store strings ignored when truncating
        return_offsets_mapping=True, #stores the location of tokens in original text
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [9]:
#use previously defined function to preprocess training data
train_dataset = raw_datasets["train"].map(
    preprocess_training_examples,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)
len(raw_datasets["train"]), len(train_dataset) #check the sentences have been tokenised

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

[codecarbon INFO @ 17:45:57] Energy consumed for RAM : 0.000048 kWh. RAM Power : 5.789747714996338 W
[codecarbon INFO @ 17:45:57] Energy consumed for all GPUs : 0.000053 kWh. Total GPU Power : 7.047371845037697 W
[codecarbon INFO @ 17:45:57] Energy consumed for all CPUs : 0.000355 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 17:45:57] 0.000456 kWh of electricity used since the beginning.


(87599, 88520)

In [10]:
#similar to training data but with simplified offset mapping, use of example ids and not the start and end positions,
def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [11]:
#use function to preprocess validation data
validation_dataset = raw_datasets["validation"].map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=raw_datasets["validation"].column_names,
)
len(raw_datasets["validation"]), len(validation_dataset)

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

(10570, 10792)

post processing

In [12]:
#initialise new/untrained model
from transformers import BertConfig, BertForQuestionAnswering
config = BertConfig.from_pretrained("bert-base-cased")
config.vocab_size = len(tokenizer)
model = BertForQuestionAnswering(config)


[codecarbon INFO @ 17:46:12] Energy consumed for RAM : 0.000072 kWh. RAM Power : 5.789747714996338 W
[codecarbon INFO @ 17:46:12] Energy consumed for all GPUs : 0.000090 kWh. Total GPU Power : 8.74668487397398 W
[codecarbon INFO @ 17:46:12] Energy consumed for all CPUs : 0.000531 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 17:46:12] 0.000694 kWh of electricity used since the beginning.


In [13]:
from tqdm.auto import tqdm
import collections
import numpy as np

n_best = 20
max_answer_length = 30
predicted_answers = []

import evaluate

metric = evaluate.load("squad")

def get_example(index):
    return tokenized_datasets["test"][index]["text"]

def compute_metrics(start_logits, end_logits, features, examples):
    experiment = comet_ml.get_global_experiment()
    example_to_features = collections.defaultdict(list)

    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []

    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples
    ]
    metric_result = metric.compute(predictions=predicted_answers, references=theoretical_answers)
    if experiment:
        #experiment.set_epoch(epoch)
        experiment.log_metric("f1", metric_result['f1'])
        experiment.log_metric("exact_match", metric_result['exact_match'])
    return metric_result

Training

In [14]:
import transformers
from transformers import TrainingArguments
import accelerate

%env COMET_MODE=ONLINE
%env COMET_LOG_ASSETS=TRUE

args = TrainingArguments(
    "no-pretraining-bert",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=15, #increase epochs
    weight_decay=0.01,
    logging_steps=100,
    fp16=True,
    push_to_hub=True,
    report_to=["comet_ml"],
    logging_strategy="epoch",
    seed=42,
    load_best_model_at_end=False,
    save_total_limit=1,
)

env: COMET_MODE=ONLINE
env: COMET_LOG_ASSETS=TRUE


In [15]:
from transformers import Trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(
[codecarbon INFO @ 17:46:15] Energy consumed for RAM : 0.000078 kWh. RAM Power : 5.789747714996338 W
[codecarbon INFO @ 17:46:15] Energy consumed for all GPUs : 0.000101 kWh. Total GPU Power : 12.374810897703151 W
[codecarbon INFO @ 17:46:15] Energy consumed for all CPUs : 0.000570 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 17:46:16] 0.000749 kWh of electricity used since the beginning.
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ------------------------------------

Epoch,Training Loss,Validation Loss
1,4.185700,No log
2,3.714600,No log
3,3.525100,No log
4,3.374000,No log
5,3.216900,No log
6,3.042100,No log
7,2.834800,No log
8,2.599000,No log
9,2.352100,No log
10,2.111200,No log


[codecarbon INFO @ 17:46:34] Energy consumed for RAM : 0.000024 kWh. RAM Power : 5.789747714996338 W
[codecarbon INFO @ 17:46:34] Energy consumed for all GPUs : 0.000105 kWh. Total GPU Power : 25.139042324073763 W
[codecarbon INFO @ 17:46:34] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 17:46:34] 0.000306 kWh of electricity used since the beginning.
[codecarbon INFO @ 17:46:49] Energy consumed for RAM : 0.000048 kWh. RAM Power : 5.789747714996338 W
[codecarbon INFO @ 17:46:49] Energy consumed for all GPUs : 0.000583 kWh. Total GPU Power : 114.7528509343659 W
[codecarbon INFO @ 17:46:49] Energy consumed for all CPUs : 0.000354 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 17:46:49] 0.000985 kWh of electricity used since the beginning.
[codecarbon INFO @ 17:47:04] Energy consumed for RAM : 0.000072 kWh. RAM Power : 5.789747714996338 W
[codecarbon INFO @ 17:47:04] Energy consumed for all GPUs : 0.001069 kWh. Total GPU Power : 116.799106230234

TrainOutput(global_step=165975, training_loss=2.576878542043229, metrics={'train_runtime': 16810.5793, 'train_samples_per_second': 78.986, 'train_steps_per_second': 9.873, 'total_flos': 2.602123751955456e+17, 'train_loss': 2.576878542043229, 'epoch': 15.0})

In [16]:
predictions, _, _ = trainer.predict(validation_dataset)
start_logits, end_logits = predictions
compute_metrics(start_logits, end_logits, validation_dataset, raw_datasets["validation"])

  0%|          | 0/10570 [00:00<?, ?it/s]

{'exact_match': 10.085146641438032, 'f1': 19.069814452913338}

In [17]:
trainer.push_to_hub(commit_message="Training complete")

'https://huggingface.co/noushsuon/no-pretraining-bert/tree/main/'

In [18]:
from torch.utils.data import DataLoader
from transformers import default_data_collator, DataCollatorForLanguageModeling

train_dataset.set_format("torch")
validation_set = validation_dataset.remove_columns(["example_id", "offset_mapping"])
validation_set.set_format("torch")

train_dataloader = DataLoader(
    train_dataset,
    shuffle=True,
    collate_fn=default_data_collator,
    batch_size=8,
)

eval_dataloader = DataLoader(
    validation_set,
    collate_fn=default_data_collator,
    batch_size=8
)

In [19]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

In [20]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [21]:
from transformers import get_scheduler

num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [22]:
from huggingface_hub import Repository, get_full_repo_name
model_name = "no-pretraining-bert"
repo_name = get_full_repo_name(model_name)
repo_name

'noushsuon/no-pretraining-bert'

In [23]:
from huggingface_hub import Repository
model_checkpoint = 'Untrained-BERT'
output_dir = "no-pretraining-bert-accelerate" + model_checkpoint

repo = Repository(output_dir, clone_from=repo_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/noushsuon/no-pretraining-bert into local empty directory.


Download file model.safetensors:   0%|          | 1.45k/414M [00:00<?, ?B/s]

Download file training_args.bin: 100%|##########| 4.43k/4.43k [00:00<?, ?B/s]

Clean file training_args.bin:  23%|##2       | 1.00k/4.43k [00:00<?, ?B/s]

Clean file model.safetensors:   0%|          | 1.00k/414M [00:00<?, ?B/s]

In [24]:
#finetune cycle
from tqdm.auto import tqdm
import torch

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for step, batch in enumerate(train_dataloader):
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    start_logits = []
    end_logits = []
    accelerator.print("Evaluation!")
    for batch in tqdm(eval_dataloader):
        with torch.no_grad():
            outputs = model(**batch)

        start_logits.append(accelerator.gather(outputs.start_logits).cpu().numpy())
        end_logits.append(accelerator.gather(outputs.end_logits).cpu().numpy())

    start_logits = np.concatenate(start_logits)
    end_logits = np.concatenate(end_logits)
    start_logits = start_logits[: len(validation_dataset)]
    end_logits = end_logits[: len(validation_dataset)]

    metrics = compute_metrics(
        start_logits, end_logits, validation_dataset, raw_datasets["validation"]
    )

    for key, value in metrics.items():
        experiment.log_metric(key, value)

    print(f"epoch {epoch}:", metrics)

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)
        repo.push_to_hub(
            commit_message=f"Training in progress epoch {epoch}", blocking=False
        )

  0%|          | 0/33195 [00:00<?, ?it/s]

Evaluation!


  0%|          | 0/1349 [00:00<?, ?it/s]

  0%|          | 0/10570 [00:00<?, ?it/s]

epoch 0: {'exact_match': 9.78240302743614, 'f1': 18.504917633535307}
Evaluation!


...

  0%|          | 0/10570 [00:00<?, ?it/s]

epoch 1: {'exact_match': 10.501419110690634, 'f1': 19.16407961298335}
Evaluation!


  0%|          | 0/1349 [00:00<?, ?it/s]

  0%|          | 0/10570 [00:00<?, ?it/s]

epoch 2: {'exact_match': 10.510879848628193, 'f1': 19.59714955541422}


In [25]:
accelerator.wait_for_everyone()
unwrapped_model = accelerator.unwrap_model(model)
unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)

In [26]:
from transformers import pipeline

model_checkpoint = "noushsuon/no-pretraining-bert"
question_answerer = pipeline("question-answering", model=model_checkpoint)

config.json:   0%|          | 0.00/633 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/434M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/217k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/688k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [27]:
context = """
Bill is Andrew's father
"""
question = "Who is Bill's father?"
answer = question_answerer(question=question, context=context)
print(context, question)
print(answer)


Bill is Andrew's father
 Who is Bill's father?
{'score': 0.8582582473754883, 'start': 1, 'end': 15, 'answer': 'Bill is Andrew'}


In [28]:
context = """
Bill is Andrew's father
"""
question = "Who is Andrew's son?"
answer = question_answerer(question=question, context=context)
print(context, question)
print(answer)


Bill is Andrew's father
 Who is Andrew's son?
{'score': 0.4675098955631256, 'start': 1, 'end': 15, 'answer': 'Bill is Andrew'}


In [29]:
experiment.end()